In [31]:

import os

WEIGHTS_DIR = './weights'
ORIGINAL_TRAIN_DIR = os.path.join('../datasets/poles_lem_vdc', 'train')
REDUCED_TRAIN_DIR = os.path.join('../datasets/poles_lem_vdc', 'train_reduced')
REDUCED_VAL_DIR = os.path.join('../datasets/poles_lem_vdc', 'val_reduced')

LEM = os.path.join('../datasets/poles_lem_vdc', 'lem')
VCA = os.path.join('../datasets/poles_lem_vdc', 'vca')

In [36]:
# Data preparation

import glob
import random
import shutil

if not os.path.isdir(WEIGHTS_DIR):
    os.makedirs(WEIGHTS_DIR)

if not os.path.isdir(REDUCED_TRAIN_DIR):
    os.makedirs(REDUCED_TRAIN_DIR)

if not os.path.isdir(REDUCED_VAL_DIR):
    os.makedirs(REDUCED_VAL_DIR)
        
N_train = 1000
N_val = 400

print 'waiting...'

if os.path.isdir(REDUCED_TRAIN_DIR):
        # if not exists, create folder "luminous" into the:
        if not os.path.isdir(REDUCED_TRAIN_DIR + "/luminous"):
            os.makedirs(REDUCED_TRAIN_DIR + "/luminous")
            
        # if not exists, create folder "luminous" into the:
        if not os.path.isdir(REDUCED_VAL_DIR + "/luminous"):
            os.makedirs(REDUCED_VAL_DIR + "/luminous")
        
        dst_train_dir = os.path.join(REDUCED_TRAIN_DIR, 'luminous')
        dst_val_dir = os.path.join(REDUCED_VAL_DIR, 'luminous')
        
        # clear files into folder:
        files = glob.glob(dst_train_dir + "/*")
        for f in files:
            os.remove(f)
        
        # clear files into folder:          
        files = glob.glob(dst_val_dir + "/*")
        for f in files:
            os.remove(f)
            
        # Created array/slice for manager dataset.
        filenameLEM = [];
        filenameVCA = [];
        
        # Get all names of files and put on into the "filenameLEM" and "filenameVCA"
        for folder, subs, files in os.walk(LEM):
            for basename in files:
                filenameLEM.append(os.path.join(folder, basename))
                
        for folder, subs, files in os.walk(VCA):
            for basename in files:
                filenameVCA.append(os.path.join(folder, basename))
        
        # Copy files for new folders.
        for f in filenameLEM[:N_train]:
            shutil.copy(f, dst_train_dir)

        for f in filenameVCA[N_train:N_train + N_val]:
            shutil.copy(f, dst_val_dir)
         
print 'finish!'

waiting...
finish!


In [4]:
import keras
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.optimizers import SGD
import numpy as np
print(keras.__version__)

2.1.3


In [5]:
base_model = VGG16(weights='imagenet', include_top=False)

# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)

# Add a binary classification layer (sigmoid)
predictions = Dense(1, activation='sigmoid')(x)

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [6]:
def f1_score(y_true, y_pred):

    # Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c2 = K.sum(K.round(K.clip(y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

    # If there are no true samples, fix the F1 score at 0.
    if c3 == 0:
        return 0

    # How many selected items are relevant?
    precision = c1 / c2

    # How many relevant items are selected?
    recall = c1 / c3

    # Calculate f1_score
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

In [7]:
from keras import backend as K

def mcor(y_true, y_pred):
    #matthews_correlation
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos


    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos


    tp = K.sum(y_pos * y_pred_pos)
    tn = K.sum(y_neg * y_pred_neg)


    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg)


    numerator = (tp * tn - fp * fn)
    denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

    return numerator / (denominator + K.epsilon())

def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))

In [8]:
# keras.optimizers.Nadam(lr=1e-6, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)

# model.compile(loss='binary_crossentropy',
#               optimizer='adam', 
#               metrics=['accuracy',f1_score])

# model.compile(loss='categorical_crossentropy',
#               optimizer='adadelta',
#               metrics=['accuracy', 'f1score', 'precision', 'recall'])


#you can use it like this
model.compile(loss='binary_crossentropy',
              optimizer= "adam",
              metrics=[mcor,recall, f1])

# model.compile(optimizer=SGD(lr=0.00001, momentum=0.9), loss='binary_crossentropy', metrics=['accuracy', f1_score])

In [9]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input

height, width = 224, 224
datagen = ImageDataGenerator(
    rotation_range=45.,
    width_shift_range=0.05,
    height_shift_range=0.05,
    shear_range=0.05,
    zoom_range=0.1,
    fill_mode='reflect',
    horizontal_flip=True,
    vertical_flip=False,
    preprocessing_function=preprocess_input)

batch_size = 15

print REDUCED_TRAIN_DIR

train_generator = datagen.flow_from_directory(
        REDUCED_TRAIN_DIR,
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='binary')

validation_generator = datagen.flow_from_directory(
        REDUCED_VAL_DIR,
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='binary')

data/train_reduced
Found 992 images belonging to 1 classes.
Found 100 images belonging to 1 classes.


In [ ]:
from keras.callbacks import TensorBoard

tb_callback = TensorBoard(log_dir='tb_log')

training = model.fit_generator(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        epochs=1,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // batch_size,
        callbacks=[tb_callback])

In [ ]:
model.save('trained_model.h5')

In [ ]:
from keras.models import load_model
model = load_model('trained_model.h5', custom_objects={'f1_score': f1_score})


In [ ]:
# Inferência em uma imagem
from keras.preprocessing import image
from skimage.io import imshow
import numpy as np

img = image.load_img('data/train/IrbKd2cK8QU1lDhfnxq9.jpg', target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
preds = model.predict(x)
print preds